In [47]:
import numpy as np
import pandas as pd
import oandapyV20
import oandapyV20.endpoints.instruments as instruments
import matplotlib.dates as mpl_dates
import matplotlib.pyplot as plt
from mpl_finance import candlestick_ohlc
import datetime
import zulu
import os

ID = '101-004-9949998-001'
token = '4900a27fe17ab573d490ce234465329f-6633f44d08bf25b48ceaacd2c7917674'

client = oandapyV20.API(access_token=token)

In [54]:
# Loop to create the dataset
Date = pd.Series(datetime.datetime(2010, 1, 1, 0, 0, 0))
eurusd = pd.DataFrame()


while Date.iloc[-1].year != 2020 or Date.iloc[-1].month != 5:
    
    last = Date.iloc[-1]
    inizio = last + datetime.timedelta(minutes=15)
    inizio = zulu.create(inizio.year, inizio.month, inizio.day, inizio.hour, inizio.minute)
    inizio = inizio.isoformat('T')+'Z'
    inizio = str(inizio[:-8] + 'Z')

    params = {
            "from": inizio,
            "granularity": "M15"}

    r = instruments.InstrumentsCandles(instrument="EUR_USD",
                                       params=params)

    rv = client.request(r)

    newDat = []
    for oo in r.response['candles']:
        newDat.append([oo['time'], oo['mid']['o'], oo['mid']['h'], oo['mid']['l'], oo['mid']['c']])
    
    newEurusd = pd.DataFrame(newDat)
    newEurusd.columns = ['Time', 'Open', 'High', 'Low', 'Close']
    newEurusd["Time"] = pd.to_datetime(newEurusd['Time'])
    Date = Date.append(newEurusd['Time'])
    eurusd = eurusd.append(newEurusd)

In [55]:
eurusd.head(5)

,Time,Open,High,Low,Close
0,2010-01-03 17:45:00,1.43120,1.43172,1.43120,1.43172
1,2010-01-03 18:00:00,1.43172,1.43425,1.43105,1.43110
2,2010-01-03 18:15:00,1.43150,1.43155,1.43130,1.43155
3,2010-01-03 18:30:00,1.43175,1.43240,1.43106,1.43106
4,2010-01-03 18:45:00,1.43111,1.43157,1.43106,1.43157


In [50]:
eurusd.to_csv(os.getcwd()+'\\eurusd15min.csv')
#eurusd = pd.read_csv(os.getcwd()+'\\eurusd5min.csv', index_col=0)